In [45]:
import pandas as pd
# import dummy classifier
from sklearn.dummy import DummyClassifier
# import train_test_split
from sklearn.model_selection import train_test_split

In [46]:
data = pd.read_csv('train.csv')

In [47]:
data.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,66728,21770,1,156,64.0,140,80,2,1,0,0,1,1
1,69098,21876,1,170,85.0,160,90,1,1,0,0,1,1
2,59185,23270,1,151,90.0,130,80,1,1,0,0,1,1
3,49288,19741,1,159,97.0,120,80,1,1,0,0,1,1
4,62481,18395,1,164,68.0,120,80,1,1,0,0,1,0


In [48]:
x_train, x_val, y_train, y_val = train_test_split(data.drop('cardio', axis=1), data['cardio'], test_size=0.2, random_state=0)

In [49]:
dummy_clf = DummyClassifier(strategy='most_frequent')
dummy_clf.fit(x_train, y_train)

dummy_clf.score(x_val, y_val)

0.4988392857142857

In [50]:
#substituir algortimo aqui!!!!
import xgboost as xgb

xgb_clf = xgb.XGBClassifier()
xgb_clf.fit(x_train, y_train)

xgb_clf.score(x_val, y_val)

0.7350892857142857

In [51]:
#import min max scaler, one hot encoder, and column transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
# pode deixar dessa forma, se tiver mt ruim a gente muda
var_to_be_used = ['gluc', 'cholesterol', 'ap_hi', 'age', 'ap_lo', 'weight']
numerical_features = ['ap_hi', 'ap_lo', 'age', 'weight']
categorical_features = ['gluc', 'cholesterol']

# create column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

x_train = preprocessor.fit_transform(x_train)
x_val = preprocessor.transform(x_val)

In [52]:
# import ransearchcv
from sklearn.model_selection import RandomizedSearchCV
# define the parameters
#adaptar para os modelos, verificar os parametros no site do sklearn, por ex.
params = {
    'max_depth': [3, 5, 7, 9, 11, 13, 15],
    'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3],
    'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000],
    'min_child_weight': [1, 3, 5, 7],
    'gamma': [0.0, 0.1, 0.2, 0.3, 0.4],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
}
# create the model
xgb_clf = xgb.XGBClassifier()
# create the random search cv
random_search = RandomizedSearchCV(xgb_clf, param_distributions=params, n_iter=5, scoring='roc_auc', n_jobs=-1, cv=5, verbose=3)
# fit the model
random_search.fit(x_train, y_train)

random_search.best_params_

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 2/5] END gamma=0.0, learning_rate=0.2, max_depth=13, min_child_weight=1, n_estimators=300, subsample=0.7;, score=0.750 total time= 1.0min
[CV 3/5] END gamma=0.0, learning_rate=0.2, max_depth=13, min_child_weight=1, n_estimators=300, subsample=0.7;, score=0.739 total time= 1.1min
[CV 1/5] END gamma=0.0, learning_rate=0.2, max_depth=13, min_child_weight=1, n_estimators=300, subsample=0.7;, score=0.745 total time= 1.1min
[CV 1/5] END gamma=0.0, learning_rate=0.3, max_depth=13, min_child_weight=1, n_estimators=400, subsample=0.6;, score=0.732 total time= 1.3min
[CV 3/5] END gamma=0.0, learning_rate=0.3, max_depth=13, min_child_weight=1, n_estimators=400, subsample=0.6;, score=0.723 total time= 1.4min
[CV 4/5] END gamma=0.0, learning_rate=0.3, max_depth=13, min_child_weight=1, n_estimators=400, subsample=0.6;, score=0.740 total time= 1.5min
[CV 5/5] END gamma=0.0, learning_rate=0.3, max_depth=13, min_child_weight=1, n_estimator

{'subsample': 0.9,
 'n_estimators': 900,
 'min_child_weight': 7,
 'max_depth': 3,
 'learning_rate': 0.15,
 'gamma': 0.3}

In [53]:
# train xgboost with the best params
xgb_clf = xgb.XGBClassifier(**random_search.best_params_)
xgb_clf.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.3, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.15, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=7, missing=nan, monotone_constraints=None,
              n_estimators=900, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [59]:
xgb_clf.score(x_train, y_train)

0.7513169642857143

In [54]:
xgb_clf.score(x_val, y_val)

0.7376785714285714

In [55]:
data_test = pd.read_csv('test.csv')

In [ ]:
x_test = preprocessor.transform(data_test)
y_test = xgb_clf.predict(x_test)
xgb.score

In [57]:
from sklearn.metrics import accuracy_score
accuracy_score(data_test['cardio'], y_test)

0.7278571428571429